# NBA Sleepers

## By: Carlos Villagomez & Kevin Tran

## CPSC 310, Spring 2019

## Introduction:
We are using a dataset consisting of two decades worth of NBA player data ranging from the 1996-2016 seasons on each player's seasonal statistics who was a member of a team. 

## Data Analysis:
The dataset consists of 21 attribute values including each player's demographic variables (e.g. height, weight, age, etc.), biographical details (team, draft year, round, etc.), and basic box score statistics (avg points, rebounds, assists, etc.). There are 9,561 total instances of players, but only 1,892 unique players in the dataset. Now, there were initially 51 rows of missing data, but the author of the dataset handled this by manually filling the gaps using player data from the online database, Basketball Reference.

## Classification Results:

## Conclusion:
